In [ ]:
%%bash
nrnivmodl

In [ ]:
# Download NEURON: http://www.neuron.yale.edu/neuron/download
# Download PyNeuronToolbox: https://github.com/ahwillia/PyNeuron-Toolbox

from neuron import h
import numpy as np
import pylab as plt
import scipy.linalg
from PyNeuronToolbox.record import ez_record,ez_convert
from PyNeuronToolbox.morphology import shapeplot,shapeplot_animate
from mpl_toolkits.mplot3d import Axes3D
from JSAnimation import IPython_display
from matplotlib import animation
from matplotlib.pyplot import cm
import warnings
warnings.filterwarnings('ignore', category=SyntaxWarning)
%matplotlib inline
np.random.seed(123456789)

# Load morphology and other stuff
# --> SegLists: soma[2], dend[74], dend_5[37], apic[42], axon[1]
# --> Files from Migliore & Migliore (2012)
# --> CA1 pyramidal neuron
h.load_file('stdrun.hoc')
h.xopen('ri06.hoc')
h.xopen('fixnseg.hoc')
h.xopen('5a_nogui.hoc')
h.tstop = 700.0

In [ ]:
from PyNeuronToolbox.morphology import dist_between,allsec_preorder

def dist_to_soma(segment):
    return dist_between(h,h.soma[0](0.5),segment)

#seglist in pre-order
sec_list = allsec_preorder(h)
seg_list = []
for sec in sec_list:
    locs = np.linspace(0,1,sec.nseg+2)[1:-1]
    for loc in locs:
        seg_list.append(sec(loc))
n = len(seg_list)

In [ ]:
secL=np.zeros(len(sec_list))
secD=np.zeros(len(sec_list))
segL=np.zeros(len(sec_list))
for i in range(len(sec_list)):
    sec = sec_list[i]
    secL[i]=sec.L
    secD[i]=sec.diam
    segL[i]=sec.L/sec.nseg
    print(f"{i}: {sec.name()}, nseg={sec.nseg}, L={sec.L}, D={sec.diam}, l={sec.L/sec.nseg}, or={sec.orientation()}, hname={sec.hname()}")
segA=h.PI*secD*segL
segV=h.PI*secD**2*segL/4


In [ ]:
(min(segL),max(segL),min(secD),max(secD))

In [ ]:
x=np.array(range(len(sec_list)),dtype=int)
plt.plot(x,segL,label='length')
plt.plot(x,secD,label='diameter')
#plt.plot(x,segA,label='area')
#plt.plot(x,segV,label='volume')
plt.ylabel('Value')
plt.xlabel('Section number')
# show a legend on the plot
plt.legend()
plt.show()

In [ ]:
plt.scatter(segL,secD)
plt.xlabel('Length')
plt.ylabel('Diameter')
plt.show()

In [ ]:
plt.scatter(segL,segA)
plt.xlabel('Length')
plt.ylabel('Area')
plt.show()

In [ ]:
plt.scatter(segA,segV)
plt.xlabel('Area')
plt.ylabel('Volume')
plt.show()

In [ ]:
(dist_to_soma(sec(0)),dist_to_soma(sec(1)),sec.L,np.linspace(0,1,sec.nseg+2),np.linspace(0,1,sec.nseg+2)[1:-1])

In [ ]:
dts = [dist_to_soma(s) for s in seg_list]
dts[:11]

In [ ]:
import re
import pandas as pd
p = re.compile("^([^\\[]+)\\[.+")
tps = [p.sub("\\1",seg.sec.name()) for seg in seg_list]
pd.Series(tps).drop_duplicates().tolist()

In [ ]:
seg = seg_list[0]
(seg.node_index(),seg.x,seg.sec)

In [ ]:
(n,min(dts),max(dts))

In [ ]:
# matrix exponential used to solve linear system
from scipy.linalg import expm

# initial condition, u starts only in soma and is trafficked out to dendrites
u0 = np.zeros(2*n)
u0[0] = 1.0  # compartment 0 is the soma

In [ ]:
def snap_ss(name,clim=[0,0.0035],cmap=plt.cm.cool,title='steady-state',clabel="Color Mapping",bg='w',fs=(8,8),ext='png',show=True):
    u = utrace[n:,:]
    view = (-90,90)
    fig = plt.figure(figsize=fs,facecolor=bg)
    shapeax = plt.subplot(111, projection='3d')
    lines = shapeplot(h,shapeax,#sections=model.cell.dend,
                      cvals=u[:,-1],clim=clim,
                      cmap=cmap,order='pre',lw=2)
    #mark_locations(h,model.cell.soma[0], 0.45, color='k', ms=10)
    shapeax.view_init(*view)
    if bg=='black':
        ct = 'w'
    else:
        ct = 'black'
    plt.title(title,fontweight='bold',fontsize=14,color=ct)

    # colorbar
    sm = cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(clim[0],clim[1])) 
    plt.colorbar(sm, ax=shapeax, shrink=0.75, ticks=np.linspace(clim[0],clim[1], 7), label=clabel)
    plt.tight_layout()
    shapeax.set_facecolor(bg)
    plt.axis('off')
    #plt.savefig(name+'.'+ext)
    if not(show):
        plt.close()

In [ ]:
u0

In [ ]:
utrace = [u0]
clr = [int(i / 37) for i in range(n)]
utrace.append(np.array(clr+clr))
utrace = np.array(utrace).T
utrace.shape

In [ ]:
fig, ax = plt.subplots()
ax.scatter(dts,clr)
ax.set_xlabel('Distance to soma')
ax.set_ylabel('Segment group')

In [ ]:
from PyNeuronToolbox.morphology import get_section_path, interpolate_jagged
x=[]
y=[]
z=[]
i=0
for sec in sec_list:
    xyz = get_section_path(h,sec)
    seg_paths = interpolate_jagged(xyz,sec.nseg)
    for (j,path) in enumerate(seg_paths):
        x.append(path[0,0])
        y.append(path[0,1])
        z.append(path[0,2])
        i += 1
    
(i,(min(x),max(x)),(min(y),max(y)),(min(z),max(z)))

In [ ]:
#bins=np.array([-165,-127.5,-90,-52.5,-15,10,52.14285714,94.28571429,136.4285714,178.5714286,220.7142857,262.8571429,305,347.1428571,389.2857143,431.4285714,473.5714286,515.7142857,557.8571429,600])
bins=np.array([-127.5,-90,-52.5,-10,5,52.14285714,94.28571429,136.4285714,178.5714286,220.7142857,262.8571429,305,347.1428571,389.2857143,431.4285714,473.5714286,515.7142857,557.8571429])
result = np.digitize(np.array(y), bins)
(len(result),len(bins),result)

In [ ]:
(min(result),max(result),len(np.concatenate([result,result])))

In [ ]:
utrace = [u0]
utrace.append(np.concatenate([result,result]))
utrace = np.array(utrace).T
utrace.shape

In [ ]:
[result[:11],y[:11]]


In [ ]:
# snapshot of steady-state
#snap_ss('geom',clim=[0,20],cmap=plt.cm.YlOrBr_r)
snap_ss(f"CA1_segment_distribution88_075",clim=[0,max(result)],cmap=plt.cm.YlOrBr,bg='lightgray',
        title='Segment distribution',clabel='Segment number')
#plt.axis('on')

In [ ]:
#['soma', 'dend_5', 'apic', 'dend', 'axon']
u1=np.zeros(2*n)
for i in range(n):
    if 'soma' == tps[i]:
        u1[i+n] = 2
        u1[i]=1

utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"CA1_soma_segments_88_075",clim=[0,max(u1)],cmap=plt.cm.YlOrBr,bg='lightgray',
        title='Soma segments',clabel='Soma')
#snap_ss('soma',clim=[0,2],cmap=plt.cm.YlOrBr)

In [ ]:
#['soma', 'dend_5', 'apic', 'dend', 'axon']
u1=-1*np.ones(2*n)
for i in range(n):
    if 'dend_5' == tps[i]:
        u1[i+n] = 2
        u1[i]= 1

utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"CA1_dend_5_segments_88_075",clim=[0,max(u1)],cmap=plt.cm.YlOrBr,bg='lightgray',
        title='dend_5 segments',clabel='dend_5')
#snap_ss('dend_5',clim=[0,2],cmap=plt.cm.YlOrBr)

In [ ]:
#['soma', 'dend_5', 'apic', 'dend', 'axon']
u1=np.zeros(2*n)
for i in range(n):
    if 'apic' == tps[i]:
        u1[i+n] = 2
        u1[i]= 1

utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"CA1_apic_segments_88_075",clim=[0,max(u1)],cmap=plt.cm.YlOrBr,bg='lightgray',
        title='Apic segments',clabel='Apic')
#snap_ss('apic',clim=[0,2],cmap=plt.cm.YlOrBr)

In [ ]:
#['soma', 'dend_5', 'apic', 'dend', 'axon']
ddd=[]
u1=np.zeros(2*n)
for i in range(n):
    if 'dend' == tps[i]:
        u1[i+n] = 2
        u1[i]= 1
        ddd.append(dts[i])

utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"CA1_dend_segments_88_075",clim=[0,max(u1)],cmap=plt.cm.YlOrBr,bg='lightgray',
        title='Dend segments',clabel='Dend')
#snap_ss('dend',clim=[0,2],cmap=plt.cm.YlOrBr)

In [ ]:
(min(u1),max(u1),min(ddd),max(ddd))

In [ ]:
#['soma', 'dend_5', 'apic', 'dend', 'axon']
u1=np.zeros(2*n)
for i in range(n):
    if 'axon' == tps[i]:
        u1[i+n] = 2
        u1[i]=1

utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"CA1_Axon_segments_86_075",clim=[0,max(u1)],cmap=plt.cm.YlOrBr,bg='lightgray',fs=(8,6),
        title='Axon segments',clabel='Axon')
#snap_ss('axon',clim=[0,2],cmap=plt.cm.YlOrBr)

In [ ]:
for sec in sec_list:
    print(f"{sec.name()}: {sec.nseg}")

Read Experimental Data
----

In [ ]:
tdf=pd.read_csv('../data/seg_mapping.csv')
abbCA1=tdf['abb']
abbT={}
segIdx={}
for i in range(n):
    abbT[abbCA1[i]] = 1+ abbT.get(abbCA1[i],0)
    ll=segIdx.get(abbCA1[i],[])
    ll.append(i)
    segIdx[abbCA1[i]] = ll

(abbCA1,abbT,segIdx.keys(),{k:len(segIdx.get(k)) for k in segIdx.keys()})

In [ ]:
expD=pd.read_csv('../data/CA1_gradient.csv')
expD

In [ ]:
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= i
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"DG_section_distribution88_075",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr,fs=(8,6),
        bg='lightgray',title='Section distribution',clabel='Section number')